# Tutorial sobre CBRkit

CBRkit es una librería en Python para el desarrollo de sistemas CBR. Se centra principalmente en la etapa de recuperación y tiene un catálogo bastante amplio de medidas de similitud. 

Principales enlaces:

- Repositorio: <https://github.com/wi2trier/cbrkit/>
- Documentación: <https://wi2trier.github.io/cbrkit/cbrkit.html>
- Tests: <https://github.com/wi2trier/cbrkit/tree/main/tests> Sirven como ejemplo de código actualizado (hay un proyecto de demo pero está obsoleto)

## Instalación

Para instalar CBRkit usamos `pip install cbrkit`. Sin embargo, CBRkit tiene dependencias opcionales que hay que instalar aparte. Si quisiésemos instalar todo usaríamos `pip install cbrkit[all]` pero se recomienda instalar solo las necesarias (para esta demo solo hemos instalado las de nlp, tal y como aparece en el archivo pyproject.toml). Las dependencias que tiene son las siguiente:

- `all`: All optional dependencies
- `api`: REST API Server
- `cli`: Command Line Interface (CLI)
- `eval`: Evaluation tools for common metrics like `precision` and `recall`
- `graphs`: Graph libraries like `networkx` and `rustworkx`
- `llm`: Large Language Models (LLM) APIs like Ollama and OpenAI
- `nlp`: Standalone NLP tools `levenshtein`, `nltk`, `openai`, and `spacy`
- `timeseries`: Time series similarity measures like `dtw` and `smith_waterman`
- `transformers`: Advanced NLP tools based on `pytorch` and `transformers`

## Primeros pasos

### Cargar una base de casos

CBRkit tiene cargadores por defecto para distintos formatos de archivos (csv, json, toml, cml, yaml, y objetos Python en archivos con extensión .py ). Cualquiera de ellos se puede cargar usando `cbrkit.loaders.file` (que usará el cargador correspondiente). Al hacer esto, se creará una base de casos en la que para cada caso se le asignará como id un entero consecutivo comenzando en 0.

A la base de casos se puede acceder como a un diccionario, usando el id del caso. Internamente se usa [Polars](https://pola.rs/), que es una librería de dataframes similar a la de pandas.

In [1]:
import cbrkit
import os

# Cargamos la base de casos de 1000 coches usados que está en ./data
carsCasebase = cbrkit.loaders.file(os.path.join(os.path.abspath(""),"data", "cars-1k.csv"))
carsCasebase


/Users/guille/gitRepos/cbrkit-tutorial/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


polars(df=shape: (1_000, 11)
┌───────┬──────┬──────────────┬──────────────────┬───┬──────────────┬───────┬────────┬─────────────┐
│ price ┆ year ┆ manufacturer ┆ model            ┆ … ┆ transmission ┆ drive ┆ type   ┆ paint_color │
│ ---   ┆ ---  ┆ ---          ┆ ---              ┆   ┆ ---          ┆ ---   ┆ ---    ┆ ---         │
│ i64   ┆ i64  ┆ str          ┆ str              ┆   ┆ str          ┆ str   ┆ str    ┆ str         │
╞═══════╪══════╪══════════════╪══════════════════╪═══╪══════════════╪═══════╪════════╪═════════════╡
│ 30590 ┆ 2020 ┆ subaru       ┆ brz limited      ┆ … ┆ other        ┆ rwd   ┆ coupe  ┆ white       │
│       ┆      ┆              ┆ coupe 2d         ┆   ┆              ┆       ┆        ┆             │
│ 17500 ┆ 2013 ┆ ram          ┆ 2500             ┆ … ┆ automatic    ┆ 4wd   ┆ pickup ┆ white       │
│ 7800  ┆ 2010 ┆ honda        ┆ accord           ┆ … ┆ automatic    ┆ fwd   ┆ sedan  ┆ black       │
│ 37990 ┆ 2016 ┆ toyota       ┆ tundra double    ┆ … ┆ other  

### Definir funciones de similitud

La definición de la función de similitud se basa en la definición de una función de similitud global, así como las funciones de similitud locales para cada uno de los atributos que se quieran utilizar. Aunque tiene distintas formas de definir las funciones de similitud, vamos a comenzar con las más sencillas, que son las que se usan en casos del tipo atributo-valor (`cbrkit.sim.attribute_value`):

- La función de similitud global se define en el `cbrkit.sim.aggregator`. Con el parámetro `pooling` definimos qué función queremos utilizar [de entre un catálogo que tiene CBRkit](https://wi2trier.github.io/cbrkit/cbrkit/sim/pooling.html): mean, fmean, geametric_mean, median, max...
- Para cada uno de los atributos definimos la función de similitud local que queremos aplicar. De nuevo, CBRkit proporciona [un gran catálogo de funciones de similitud](https://wi2trier.github.io/cbrkit/cbrkit/sim.html) para distintos tipos de datos: [números](https://wi2trier.github.io/cbrkit/cbrkit/sim/numbers.html), [cadenas](https://wi2trier.github.io/cbrkit/cbrkit/sim/strings.html), [colecciones](https://wi2trier.github.io/cbrkit/cbrkit/sim/collections.html), [grafos](https://wi2trier.github.io/cbrkit/cbrkit/sim/graphs.html), [embeddings (vectores)](https://wi2trier.github.io/cbrkit/cbrkit/sim/embed.html), taxonomías...




In [2]:
# Definimos una función de similitud para la base de casos de coches
simFunction = cbrkit.sim.attribute_value(
    attributes={
        # Para el año usamos una función de similitud lineal
        # (entre un mínimo y un máximo, de acuerdo a los datos de la base de casos)
        "year": cbrkit.sim.numbers.linear_interval(1950, 2021),

        # Para el manufacturer, utilizamos la comparación de cadenas usando la distancia de Levenshtein
        "manufacturer": cbrkit.sim.strings.levenshtein(),
    },
    # Como medida de similitud global utilizamos la media 
    aggregator=cbrkit.sim.aggregator(pooling="mean"),
)

### Recuperación de casos

Para poder ejecutar la fase CBR de recuperación necesitamos:
- Configurar una estrategia de recuperación o `retriever`
- Definir una consulta: un diccionario con los atributos y valores que necesitemos. Una consulta puede ser un caso de la base de casos.
- Ejecutar la consulta usando `cbrkit.retrieval.apply_query`.

El resultado de la ejecución se almacena en un objeto de tipo [Result](https://github.com/wi2trier/cbrkit/blob/main/src/cbrkit/model/result.py). Los atributos más importantes son los siguientes:

- `result.similarities`: Contiene el cálculo de similitud de la consulta con el resto de casos en la base de casos. Produce información tanto de la similitud global como de las similitudes locales. 
- `result.ranking`: una secuencia con los ids de los casos ordenados por similitud decreciente.
- `result.duration`: tiempo que ha tardado en ejecutarse la consulta. 


In [3]:
# Creamos el retriever
retriever = cbrkit.retrieval.build(simFunction)
# Creamos una consulta
query = {"year": 2000, "manufacturer": "mazda"}
# Ejecutamos la consulta
result = cbrkit.retrieval.apply_query(carsCasebase, query, retriever)

# Revisamos los valores de similitud
result.similarities

{0: AttributeValueSim(value=0.45006402048655564, attributes={'year': 0.7183098591549295, 'manufacturer': 0.18181818181818177}),
 1: AttributeValueSim(value=0.5334507042253521, attributes={'year': 0.8169014084507042, 'manufacturer': 0.25}),
 2: AttributeValueSim(value=0.6295774647887324, attributes={'year': 0.8591549295774648, 'manufacturer': 0.4}),
 3: AttributeValueSim(value=0.4782330345710627, attributes={'year': 0.7746478873239436, 'manufacturer': 0.18181818181818177}),
 4: AttributeValueSim(value=0.5204865556978233, attributes={'year': 0.8591549295774648, 'manufacturer': 0.18181818181818177}),
 5: AttributeValueSim(value=0.5266040688575899, attributes={'year': 0.8309859154929577, 'manufacturer': 0.2222222222222222}),
 6: AttributeValueSim(value=0.5123239436619718, attributes={'year': 0.7746478873239436, 'manufacturer': 0.25}),
 7: AttributeValueSim(value=0.6577464788732394, attributes={'year': 0.9154929577464789, 'manufacturer': 0.4}),
 8: AttributeValueSim(value=0.4154929577464789

In [4]:
# Ids de los casos, en orden decreciente de similitud
result.ranking

(248,
 138,
 638,
 984,
 328,
 97,
 249,
 188,
 474,
 794,
 460,
 433,
 954,
 365,
 765,
 158,
 897,
 609,
 546,
 854,
 630,
 498,
 387,
 266,
 475,
 604,
 830,
 778,
 369,
 285,
 321,
 7,
 868,
 242,
 355,
 96,
 590,
 607,
 747,
 105,
 75,
 489,
 600,
 930,
 939,
 763,
 759,
 25,
 364,
 394,
 523,
 2,
 257,
 492,
 507,
 671,
 598,
 404,
 646,
 650,
 688,
 869,
 959,
 608,
 655,
 381,
 537,
 714,
 203,
 259,
 354,
 627,
 628,
 987,
 990,
 261,
 431,
 483,
 506,
 565,
 168,
 278,
 352,
 427,
 98,
 202,
 440,
 887,
 896,
 380,
 704,
 916,
 439,
 186,
 250,
 398,
 881,
 128,
 204,
 863,
 867,
 14,
 468,
 542,
 911,
 119,
 823,
 132,
 184,
 462,
 809,
 989,
 690,
 699,
 58,
 78,
 408,
 770,
 824,
 829,
 986,
 180,
 251,
 444,
 601,
 683,
 909,
 144,
 164,
 656,
 757,
 393,
 594,
 610,
 641,
 346,
 471,
 514,
 549,
 635,
 889,
 24,
 978,
 32,
 140,
 532,
 941,
 983,
 123,
 491,
 522,
 945,
 100,
 831,
 412,
 66,
 165,
 280,
 665,
 678,
 944,
 962,
 125,
 356,
 397,
 899,
 955,
 53,
 545,
 5

In [5]:
# Podemos ver la información de caso más similar a la consulta 
print(
    "Consulta: ",
    query,
    os.linesep,
    "Similitud: ",
    result.similarities[result.ranking[0]],
    os.linesep,
    "Caso: ",
    carsCasebase[result.ranking[0]]
)

Consulta:  {'year': 2000, 'manufacturer': 'mazda'} 
 Similitud:  AttributeValueSim(value=0.9859154929577465, attributes={'year': 0.971830985915493, 'manufacturer': 1.0}) 
 Caso:  {'price': 5990, 'year': 2002, 'manufacturer': 'mazda', 'model': 'tribute', 'condition': 'good', 'fuel': 'gas', 'title_status': 'clean', 'transmission': 'automatic', 'drive': '4wd', 'type': 'SUV', 'paint_color': 'grey'}


En lugar de utilizar directamente el retriever CBRkit proporciona una clase [`Dropout`](wi2trier.github.io/cbrkit/cbrkit/retrieval.html#dropout) que aplica ordenación y filtros sobre el retreiver, de modo que facilita el proceso de recuperación de casos similares.

In [6]:
# Usamos el dropout para quedarnos con los 10 casos más similares.
retriever = cbrkit.retrieval.dropout(
    cbrkit.retrieval.build(simFunction), # A retriever
    limit=10,
)

query = {"year": 2000, "manufacturer": "mazda"}
result = cbrkit.retrieval.apply_query(carsCasebase, query, retriever)
result.similarities

{248: AttributeValueSim(value=0.9859154929577465, attributes={'year': 0.971830985915493, 'manufacturer': 1.0}),
 138: AttributeValueSim(value=0.9788732394366197, attributes={'year': 0.9577464788732395, 'manufacturer': 1.0}),
 638: AttributeValueSim(value=0.9295774647887324, attributes={'year': 0.8591549295774648, 'manufacturer': 1.0}),
 984: AttributeValueSim(value=0.9225352112676056, attributes={'year': 0.8450704225352113, 'manufacturer': 1.0}),
 328: AttributeValueSim(value=0.8873239436619718, attributes={'year': 0.7746478873239436, 'manufacturer': 1.0}),
 97: AttributeValueSim(value=0.880281690140845, attributes={'year': 0.7605633802816901, 'manufacturer': 1.0}),
 249: AttributeValueSim(value=0.880281690140845, attributes={'year': 0.7605633802816901, 'manufacturer': 1.0}),
 188: AttributeValueSim(value=0.8732394366197183, attributes={'year': 0.7464788732394366, 'manufacturer': 1.0}),
 474: AttributeValueSim(value=0.8732394366197183, attributes={'year': 0.7464788732394366, 'manufactu

## Profundizando en CBRkit

### Base de casos con id propio

Como dijimos antes, al cargar una base de casos CBRkit usa como identificadores números sucesivos empezando en 0. En realidad, podemos utilizar como base de casos cualquier diccionario en el que almacenemos cada caso como un par atributo-valor caseId: case.

Si nuestra base de casos no se organiza de esta forma pero tiene su propio atributo para representar el id del caso entonces podemos crear la base de casos "a mano", como se puede ver en el siguiente ejemplo.


In [ ]:
import json

with open(os.path.join(os.path.abspath(""),"data",'dmhItems.json')) as f:
    dataFile = json.load(f)

# Este dataset está en un formato que se usará para visualizarlo en SimViz por lo que la base de casos está en cases
cases = dataFile["cases"]

# Este formato también almacena información de cuál es el atributo que hace las veces de caseId
attributeId = dataFile["metadata"]["id"]

# Creamos la base de casos utilizando el atributo que representa el caseId
dmhCasebase = { case[attributeId]: case for case in cases }
dmhCasebase

{'c44171': {'_id': '634f19096c70ef5cf152f237',
  'title': 'Scandia',
  'Object': 'CUTLERY',
  'Special name': '*',
  'id': 'c44171',
  'author': 'Franck, Kaj',
  'Production date': '1952',
  'Collection': 'KÃ¤yttÃ¶kokoelma',
  'Manufacturer': 'Hackman Sorsakoski',
  'Dimension in cm': 'fork: 17.5 cm (length) x 0.15cm (thickness)  knife: 18.6 cm (length) x 0.3/0.15 cm (thickness)  spoon: 18 cm (length)',
  'Weight in kg': '*',
  'Materials': ['metal'],
  'ColorName': ['metal'],
  '_datasetid': '0daa0287-d7f4-4f03-a068-95f43afcc347',
  '_timestamp': 1668599056,
  '_timestamp_year': 2022,
  '_timestamp_month': 11,
  '_timestamp_day': 16,
  '_timestamp_hour': 11,
  '_timestamp_minute': 44,
  '_timestamp_second': 16,
  '_updated': 1.0,
  'year': 1952.0,
  'Color': [[109, 114, 120]],
  'image': 'https://i.postimg.cc/Zqn1M35m/44171.png',
  'Object group': 'cutlery',
  'ColorDict': [{'colorName': 'metal', 'rgb': [109, 114, 120]}]},
 'c44168': {'_id': '634f1908b7693f159a62e2f6',
  'title': 'Sav

### Creación de funciones de similitud local

Podemos crear nuestras propias funciones de similitud local. Si no necesitamos ningún tipo de parámetro de configuración entonces nos basta con crear una función que recibe dos parámetros y devuelve un float.




In [ ]:
# Mi propia función de igualdad
myEquality = lambda x,y: 1 if x==y else 0

simFunction = cbrkit.sim.attribute_value(
    attributes={
        # Para el atributo Object usamos la igualdad
        "Object": myEquality
    },
    # Como medida de similitud global utilizamos la media 
    aggregator=cbrkit.sim.aggregator(pooling="mean"),
)

# Usamos el dropout para quedarnos con los 10 casos más similares.
retriever = cbrkit.retrieval.dropout(
    cbrkit.retrieval.build(simFunction), # A retriever
    limit=10,
)

query = {"Object": "CUTLERY"}
result = cbrkit.retrieval.apply_query(dmhCasebase, query, retriever)
result.similarities

{'c44171': AttributeValueSim(value=1.0, attributes={'Object': 1}),
 'c44168': AttributeValueSim(value=1.0, attributes={'Object': 1}),
 'c14219': AttributeValueSim(value=0.0, attributes={'Object': 0}),
 'cB561': AttributeValueSim(value=0.0, attributes={'Object': 0}),
 'c44166': AttributeValueSim(value=0.0, attributes={'Object': 0}),
 'c16850': AttributeValueSim(value=0.0, attributes={'Object': 0}),
 'c13906': AttributeValueSim(value=0.0, attributes={'Object': 0}),
 'c17876': AttributeValueSim(value=0.0, attributes={'Object': 0}),
 'c32628': AttributeValueSim(value=0.0, attributes={'Object': 0}),
 'c32627': AttributeValueSim(value=0.0, attributes={'Object': 0})}

Si la función requiere de parámetros de inicialización entonces lo mejor es crear una clase:
- Que hereda de `SimFunc[<Tipo_Atributo>, float]`
- Que en el constructor se pasen los parámetros de configuración
- Que tenga un método `__call__(self, x: <Tipo_Atributo>, y: <Tipo_Atributo>) -> float` en el que se compute la similitud.

A modo de ejemplo vamos a definir una función de similitud entre décadas para el atributo años. Así tendremos que dos casos son similares si el año de creación de ambos pertenece a la misma década. Además, queremos que el cálculo de la distancia entre décadas esté normalizado en cuanto a una década mínima y máxima. 

In [12]:
from cbrkit.typing import SimFunc
type Number = float | int

class decade_similarity(SimFunc[Number, float]):
    """Similarity based on the _Decade_ that the year belongs to, normalized in the range of
    decades included in the dataset. The _Decade_ is computed using the year and starts in "1"
    and finishes in "0" (e.g. 1950 belongs to the 40s)
    """

    def __init__(self,minYear:int, maxYear:int):
        super().__init__()
        self.minDecade = self.convertYearToDecade(minYear)
        self.maxDecade = self.convertYearToDecade(maxYear)
        self.decadeRange = self.maxDecade - self.minDecade

    def __call__(self, x: Number, y: Number) -> float:
        if (x is not None) and (y is not None):
            decade1 = self.convertYearToDecade(x)
            decade2 = self.convertYearToDecade(y)

            normalizeDecade1 = (decade1 - self.minDecade) / self.decadeRange
            normalizeDecade2 = (decade2 - self.minDecade) / self.decadeRange
            return 1-abs(normalizeDecade2 - normalizeDecade1)
        else:
            return 0.0
    
    def convertYearToDecade(self,year):
        return int((year-1)/10) * 10

In [16]:
# Recogemos los años que aparecen en la base de casos...
years = [item["year"] for item in cases if item["year"] is not None]



simFunction = cbrkit.sim.attribute_value(
    attributes={
        # Para el atributo Object usamos la igualdad
        "Object": myEquality,
        # Para el atributo year usamos la similitud entre décadas, 
        # usando el mínimo y el máximo de los años que aparecen en la base de casos
        "year": decade_similarity(min(years), max(years))
    },
    # Como medida de similitud global utilizamos la media 
    aggregator=cbrkit.sim.aggregator(pooling="mean"),
)

# Usamos el dropout para quedarnos con los 10 casos más similares.
retriever = cbrkit.retrieval.dropout(
    cbrkit.retrieval.build(simFunction), # A retriever
    limit=10,
)

query = {"Object": "VASE", "year": 1955}
result = cbrkit.retrieval.apply_query(dmhCasebase, query, retriever)
result.similarities

{'c17248': AttributeValueSim(value=1.0, attributes={'Object': 1, 'year': 1.0}),
 'c41793': AttributeValueSim(value=0.9090909090909092, attributes={'Object': 1, 'year': 0.8181818181818182}),
 'cB548': AttributeValueSim(value=0.7272727272727273, attributes={'Object': 1, 'year': 0.4545454545454546}),
 'c16101': AttributeValueSim(value=0.7272727272727273, attributes={'Object': 1, 'year': 0.4545454545454546}),
 'c44171': AttributeValueSim(value=0.5, attributes={'Object': 0, 'year': 1.0}),
 'c32628': AttributeValueSim(value=0.5, attributes={'Object': 0, 'year': 1.0}),
 'c32627': AttributeValueSim(value=0.5, attributes={'Object': 0, 'year': 1.0}),
 'c44179': AttributeValueSim(value=0.5, attributes={'Object': 0, 'year': 1.0}),
 'c7762': AttributeValueSim(value=0.5, attributes={'Object': 0, 'year': 1.0}),
 'cILM7606': AttributeValueSim(value=0.5, attributes={'Object': 0, 'year': 1.0})}

In [15]:
dmhCasebase['c17248']

{'_id': '634f19009d480d2bcc4ac7a2',
 'title': 'Salmenhaara vase',
 'Object': 'VASE',
 'Special name': '*',
 'id': 'c17248',
 'author': 'Salmenhaara, Kyllikki',
 'Production date': '1957',
 'Collection': '*',
 'Manufacturer': 'Arabia / Salmenhaara, Kyllikki',
 'Dimension in cm': '9 (base diameter) x 26,5',
 'Weight in kg': '0,7',
 'Materials': ['stoneware'],
 'ColorName': ['brown'],
 '_datasetid': '0daa0287-d7f4-4f03-a068-95f43afcc347',
 '_timestamp': 1666128128,
 '_timestamp_year': 2022,
 '_timestamp_month': 10,
 '_timestamp_day': 18,
 '_timestamp_hour': 22,
 '_timestamp_minute': 22,
 '_timestamp_second': 8,
 '_updated': None,
 'year': 1957.0,
 'Color': [[165, 42, 42]],
 'image': 'https://apicollection.designmuseum.fi/wp-content/uploads/2017/02/wsi-imageoptim-am6077-2000x2667.jpg',
 'Object group': 'pots',
 'ColorDict': [{'colorName': 'brown', 'rgb': [165, 42, 42]}]}

### Peso de los atributos

Hasta ahora todos los atributos aplican lo mismo en las funciones de similitud pero podemos definir funciones en las que unos atributos sean más importantes que otros de cara a la similitud. Para esto, podemos indicar los pesos de cada atributo usando el parámetro `pooling_weights` del `aggregator`, que es un diccionario con el peso de cada uno de los atributos (los pesos no han de estar normalizados ni sumar 1 sino que pueden tener cualquier valor entero). 

In [ ]:
simFunction = cbrkit.sim.attribute_value(
    attributes={
        # Para el atributo Object usamos la igualdad
        "Object": myEquality,
        # Para el atributo year usamos la similitud entre décadas, 
        # usando el mínimo y el máximo de los años que aparecen en la base de casos
        "year": decade_similarity(min(years), max(years))
    },
    # Como medida de similitud global utilizamos la media ponderada de modo que la década tenga mucho más peso que el tipo de objeto.
    aggregator=cbrkit.sim.aggregator(pooling="mean", pooling_weights={"year": 0.95, "Object": 0.05}),
)

# Usamos el dropout para quedarnos con los 10 casos más similares.
retriever = cbrkit.retrieval.dropout(
    cbrkit.retrieval.build(simFunction), # A retriever
    limit=10,
)

query = {"Object": "VASE", "year": 1955}
result = cbrkit.retrieval.apply_query(dmhCasebase, query, retriever)

# Podemos ver que los resultados son diferentes a la anterior ejecución.
result.similarities


{'c17248': AttributeValueSim(value=1.0, attributes={'Object': 1, 'year': 1.0}),
 'c44171': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0}),
 'c32628': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0}),
 'c32627': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0}),
 'c44179': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0}),
 'c7762': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0}),
 'cILM7606': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0}),
 'c10026': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0}),
 'c12595': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0}),
 'c22769': AttributeValueSim(value=0.95, attributes={'Object': 0, 'year': 1.0})}

In [ ]:
simFunction = cbrkit.sim.attribute_value(
    attributes={
        # Para el atributo Object usamos la igualdad
        "Object": myEquality,
        # Para el atributo year usamos la similitud entre décadas, 
        # usando el mínimo y el máximo de los años que aparecen en la base de casos
        "year": decade_similarity(min(years), max(years))
    },
    # Como medida de similitud global utilizamos la media ponderada de modo que la década pese el doble que la del tipo de objeto (66% - 33%)
    aggregator=cbrkit.sim.aggregator(pooling="mean", pooling_weights={"year": 2, "Object": 1}),
)

# Usamos el dropout para quedarnos con los 10 casos más similares.
retriever = cbrkit.retrieval.dropout(
    cbrkit.retrieval.build(simFunction), # A retriever
    limit=10,
)

query = {"Object": "VASE", "year": 1955}
result = cbrkit.retrieval.apply_query(dmhCasebase, query, retriever)

# Podemos ver que los resultados son diferentes a la anterior ejecución.
result.similarities


{'c17248': AttributeValueSim(value=1.0, attributes={'Object': 1, 'year': 1.0}),
 'c41793': AttributeValueSim(value=0.8787878787878789, attributes={'Object': 1, 'year': 0.8181818181818182}),
 'c44171': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0}),
 'c32628': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0}),
 'c32627': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0}),
 'c44179': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0}),
 'c7762': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0}),
 'cILM7606': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0}),
 'c10026': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0}),
 'c12595': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0})}

### Ejecución de múltiples consultas

El módulo `cbrkit.retrieval`no solo tiene el método `apply_query`, que ejecuta una sola query, sino que tiene el método `apply_queries` para ejecutar múltiples queries (cada una identificada con un id) sobre una base de casos. Esto puede ser útil, por ejemplo, para hacer que se calculen la similitud entre todos los pares de casos en la base de casos.

En este caso, en lugar de tener un único objeto `similarities` tendremos uno por cada query. Para acceder a ellos usaremos  `result.queries[<id de la query>]`

In [ ]:
retriever = cbrkit.retrieval.build(simFunction)

# Vamos a ejecutar CBRkit para calcular todas las similitudes entre todos los casos de la base de casos
# OJO: Esta ejecución puede llevar mucho tiempo ya que se ejecutan N*(N-1)/2 cálculos de similitud (donde N es el tamaño de la base de casos)
result = cbrkit.retrieval.apply_queries(dmhCasebase, dmhCasebase, retriever)

# Para acceder a los resultados de una de las consultas (o casos) basta con usar el id de dicha consulta (o caso)
result.queries["c44171"]

QueryResultStep[TypeVar, TypeVar, TypeVar](similarities={'c44171': AttributeValueSim(value=1.0, attributes={'Object': 1, 'year': 1.0}), 'c44168': AttributeValueSim(value=0.9393939393939394, attributes={'Object': 1, 'year': 0.9090909090909091}), 'c14219': AttributeValueSim(value=0.606060606060606, attributes={'Object': 0, 'year': 0.9090909090909091}), 'cB561': AttributeValueSim(value=0.0, attributes={'Object': 0, 'year': 0.0}), 'c44166': AttributeValueSim(value=0.48484848484848475, attributes={'Object': 0, 'year': 0.7272727272727272}), 'c16850': AttributeValueSim(value=0.5454545454545454, attributes={'Object': 0, 'year': 0.8181818181818182}), 'c13906': AttributeValueSim(value=0.5454545454545454, attributes={'Object': 0, 'year': 0.8181818181818182}), 'c17876': AttributeValueSim(value=0.30303030303030304, attributes={'Object': 0, 'year': 0.4545454545454546}), 'c32628': AttributeValueSim(value=0.6666666666666666, attributes={'Object': 0, 'year': 1.0}), 'c32627': AttributeValueSim(value=0.6